# 10.3.3 Scrape Mars Data: The News - Article Scraping

In [13]:
# 10.3.3
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# 10.3.5
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\ckbau\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
# searching for div tag and attribute list_text
# wait time is 1 second - browser waits 1 sec before searching for components
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# HTML parser
# slide_elem variable to look for <div /> tag and its descendent
# div is tag and list_text is class
# select_one
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# assign title and summary text to variables - narrowing down in the website
# use .find to get specific
# we know the title has div and class of content_title
slide_elem.find('div', class_='content_title')

# output will be full div HTML

<div class="content_title">AI Is Helping Scientists Discover Fresh Craters on Mars</div>

In [6]:
# only want the text and not extra HTML stuff
# Use the parent element to find the first `a` tag and save it as `news_title`
# use .get_text to only get text of the element returned
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

# output will be just the text of the Title of most recent article

'AI Is Helping Scientists Discover Fresh Craters on Mars'

In [7]:
# find the summary of the first article which is most recent article
# Use the parent element to find the paragraph text
# .find() gets the first class and attribute with the specifics
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

# output will be just the summary of the article

"It's the first time machine learning has been used to find previously unknown craters on the Red Planet."

# 10.3.4 Scrape Mars Data: Featured Image

### Featured Images

In [8]:
# Visit URL to get the full image
url = 'https://spaceimages-mars.com'
browser.visit(url)

# will launch a new automated browser to the featured images webpage

In [9]:
# get the full image by clicking on the button on the webpage
# Find and click the full image button
# using spinter, click the second button [1]
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

# output with automated browser that will click the button and change the view to a slideshow

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
# BeautifulSoup looks inside <img /> tag for image with class of fancybox-image
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel


##### NOTE:  can't find fancybox-image in DevTools ?????? can see fancybox-thumb though
# output should be the URL of the link .....jpg

'image/featured/mars3.jpg'

In [12]:
# Use the base URL to create an absolute URL
# create a complete URL by adding base URL to the partial URL created above
# create variable img_url to hold the f string
# f string formatting to print in Python
# curly brackets hold variable inserted into the f string
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

# f strings work good because evaluated at run time.  Doesn't exist until the code is executed
#   and the values are not constant.  Works well for scraping because data is live

'https://spaceimages-mars.com/image/featured/mars3.jpg'

# 10.3.5 Scrape Mars Data: Mars Facts

In [14]:
# read table information using Pandas read_html()
# read_html() specifically searches for and returns a list of tables found in HTML
# index of 0 tells Pandas to pull only the first table it encounters

df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# convert the df back into HTML
df.to_html()

# output is a table with lots of nested elements

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
# when done scraping, turn off automated browsing session
# better for computer memory

browser.quit()